In [2]:
from database_queries import database_query
from database_session.database_session import DatabaseSession

ValueError: Unable to configure handler 'file'

In [ ]:
DATABASE_PATH = "./database/hockey_v12.db"

In [ ]:
db_session = DatabaseSession(db_path=DATABASE_PATH)

In [ ]:
query_o = database_query.DbDataGetter(session=)